In [ ]:
# Import experiment runners and utilities
from dataclasses import asdict
# Training pipeline for experiments 1 and 2
from training import train as pipeline_train, train_with_outlier_clipping, ExperimentConfig as TConfig
# DP virtual projection modules
from dp_virtual_projection_population_only import ExperimentConfig as VProjConfig, train as vp_train
# Local DP-SGD modules for other experiments
from local_dpsgd_experiment import (
    ExperimentConfig as LDConfig,
    train_local,
    train_mixup,
    train_momentum,
    apply_params,
)
# Custom experiments
from embedding_clustering_experiment import run_experiment as cluster_run
from private_lasso_experiment import run_experiment as lasso_run
# Utilities
from experiment_utils import clear_memory


In [ ]:
# 1. Baseline clipping experiment using the DP training pipeline
baseline_cfg_dict = {
    'device': 'cpu',
    'seed': 42,
    'num_epochs': 5,
    'c_start': 4.0,
    'c_end': 2.0,
    'noise_mult': 1.0,
    'delta': 1e-5,
    'inner_momentum': 0.5,
    'outer_momentum': 0.9,
    'lr': 0.05,
    'schedule_milestones': [10, 20],
    'schedule_gamma': 0.1,
    'max_momentum_size': 10000,
    'self_aug_factor': 1,
    'drop_after_frac': 0.5,
    'default_clip': 1.0,
    'outlier_clip': 0.5,
    'high_err_threshold': 0.1,
}
baseline_cfg_dict  # Display adjustable parameters
baseline_cfg = TConfig(**baseline_cfg_dict)
baseline_results = pipeline_train(baseline_cfg, 'outputs/baseline_clipping')
clear_memory()
baseline_results


In [ ]:
# 2. Outlier clipping experiment using the DP training pipeline
outlier_cfg_dict = {
    'device': 'cpu',
    'seed': 42,
    'num_epochs': 5,
    'c_start': 4.0,
    'c_end': 2.0,
    'noise_mult': 1.0,
    'delta': 1e-5,
    'inner_momentum': 0.5,
    'outer_momentum': 0.9,
    'lr': 0.05,
    'schedule_milestones': [10, 20],
    'schedule_gamma': 0.1,
    'max_momentum_size': 10000,
    'self_aug_factor': 1,
    'drop_after_frac': 0.3,
    'default_clip': 1.0,
    'outlier_clip': 0.5,
    'high_err_threshold': 0.2,
}
outlier_cfg_dict  # Display adjustable parameters
outlier_cfg = TConfig(**outlier_cfg_dict)
outlier_results = train_with_outlier_clipping(outlier_cfg, 'outputs/outlier_clipping')
clear_memory()
outlier_results


In [ ]:
# 3. Virtual samples experiment using DP virtual projection
vp_cfg = VProjConfig(
    num_epochs=1,
    batch_size=256,
    lr=0.05,
    outer_momentum=0.9,
    inner_momentum=0.10,
    noise_mult=1.0,
    delta=1e-5,
    c_start=4.0,
    c_end=2.0,
    self_aug_factor=1,
    schedule_milestones=[12, 18],
    schedule_gamma=0.1,
    max_momentum_size=10000,
    seed=42,
    device='cpu',
)
vp_cfg_dict = asdict(vp_cfg)  # Display adjustable parameters
vp_cfg_dict
vp_results = vp_train(vp_cfg)
clear_memory()
vp_results


In [ ]:
# 4. Local DP-SGD with synthetic public data (DP mix diff)
dp_mix_diff_cfg = asdict(LDConfig())
# Override and explicitly set key parameters for DP mix diff experiment
dp_mix_diff_cfg.update({
    'EXPERIMENT': 'local_pub',
    'NUM_EPOCHS': 5,
    'C_CLIP_PRIV': 4.0,
    'C_CLIP_PUB': 4.0,
    'NOISE_MULT': 1.0,
    'DELTA': 1e-5,
    'MOMENTUM_LOCAL': 0.0,
    'PUBLIC_METHOD': 'add',
    'SELF_AUG_FACTOR': 1,
})
dp_mix_diff_cfg  # Display adjustable parameters
apply_params(dp_mix_diff_cfg)
dp_mix_diff_results = train_mixup(use_public=True, mode='diff')
clear_memory()
dp_mix_diff_results


In [ ]:
# 5. Private LASSO experiment
lasso_params = {
    'data_fraction': 0.1,
    'num_iterations': 30,
    'noise_mult': 1.0,
    'delta': 1e-5,
}
lasso_params  # Display adjustable parameters
lasso_results = lasso_run(lasso_params)
clear_memory()
lasso_results


In [ ]:
# 6. DP mix experiment (private mixup)
dp_mix_self_cfg = asdict(LDConfig())
# Override and explicitly set key parameters for DP mix self experiment
dp_mix_self_cfg.update({
    'EXPERIMENT': 'local_priv',
    'NUM_EPOCHS': 5,
    'C_CLIP_PRIV': 4.0,
    'NOISE_MULT': 1.0,
    'DELTA': 1e-5,
    'MOMENTUM_LOCAL': 0.0,
    'SELF_AUG_FACTOR': 1,
})
dp_mix_self_cfg  # Display adjustable parameters
apply_params(dp_mix_self_cfg)
dp_mix_self_results = train_mixup(use_public=False, mode='self')
clear_memory()
dp_mix_self_results


In [ ]:
# 7. Momentum based local DP-SGD
momentum_cfg_dict = asdict(LDConfig())
# Override and explicitly set key parameters for momentum experiment
momentum_cfg_dict.update({
    'EXPERIMENT': 'local_priv',
    'NUM_EPOCHS': 5,
    'C_CLIP_PRIV': 4.0,
    'NOISE_MULT': 1.0,
    'DELTA': 1e-5,
    'MOMENTUM_LOCAL': 0.9,
})
momentum_cfg_dict  # Display adjustable parameters
apply_params(momentum_cfg_dict)
momentum_results = train_momentum(use_public=False)
clear_memory()
momentum_results


In [ ]:
# 8. Clustering embedding experiment
cluster_params = {
    'data_fraction': 0.1,
    'n_synth_per_class': 500,
    'hidden_dim': 64,
    'noise_mult': 1.0,
    'delta': 1e-5,
}
cluster_params  # Display adjustable parameters
cluster_results = cluster_run(cluster_params)
clear_memory()
cluster_results
